# Welcome

Hi! this code shows and explains Debye-Gruneisen model calculations for a given system Energy vs Volume (EV) curve. You will see how the shape of the curve (and the speed of sound) effect the thermodynamic properties of solids. Specifically you will calculate the entropy, heat capcity, and Helmholtz free energy. Step by step explainations will be given so that you can see the inner workings of the Debye-Gruneisen model.

We will predominately use code from the **D**ensity **F**unctional **T**ool**k**it (DFTTK) written by Luke A Myers and the Dr. Nigel Hew. DFTTK can be used to calculate thermodynamic properties from first principles. However, this notebook will focus only on calculations from a given EV curve defined by the user using the four parameter Birch Murnaghan equation of state.


In [12]:
# import the necessary packages 
# should take less than 30 seconds

import numpy as np
import plotly.graph_objs as go
from scipy import constants
from scipy.special import bernoulli, gamma

In [2]:
# definitions

"""
Debye Model
"""

# The definition of the Debye temperature is A * hbar/k_B * N/V) where A is
# A = (6 * pi^2)^1/3 * hbar/k_B = 2.977*10^-11 s*K
# converting to the of the eos_parameters,
# A = (hbar/kb) * (6*math.pi**2)**(1/3) * (1*10**-10 m/Å)**(1/2) * (1*10**12 (g/(ms^2))/GPa)**(1/2) / ((1.66054*10**-24 g/u)**(1/2))
# A = 231.0389521318254 K/(Å*GPa/u)^1/2
A = 231.0389521318254
BOLTZMANN_CONSTANT = constants.physical_constants["Boltzmann constant in eV/K"][0]

def gruneisen_parameter(bulk_modulus_prime: float, gruneisen_x: float) -> float:
    """Calculates the gruneisen parameter, gamma

    Args:
        bulk_modulus_prime: B_0', first derivative of the bulk modulus with respect to pressure
        gruneisen_x: Typical values include x = 2/3 (high temperature) and x = 1 (low temperature)

    Returns:
        gamma, the gruneisen parameter
    """
    return (1 + bulk_modulus_prime) / 2 - gruneisen_x

def debye_temperature(
    volume: float,
    eos_parameters: tuple[float],
    mass: float,
    gru_param: float,
    scaling_factor: float = 0.617,
) -> float:
    """Calculates the Debye temperature within the Debye-Gruneisen model.

    Args:
        volume: Volume of the cell
        eos_parameters: The parameters of the equation of state: (volume_0, energy_0, bulk_modulus, bulk_modulus_prime)
        mass: Total mass of the cell
        gru_param: gruneisen parameter
        scaling_factor: The scaling factor defaults to 0.617, as determined by Moruzzi et al. from their study on
        nonmagnetic cubic metals (https://doi.org/10.1103/PhysRevB.37.790).

    Returns:
        Debye temperature
    """

    s = scaling_factor
    volume_0 = eos_parameters[0]
    bulk_modulus = eos_parameters[2]

    return (
        s
        * A
        * volume_0 ** (1 / 6)
        * (bulk_modulus / mass) ** (1 / 2)
        * (volume_0 / volume) ** gru_param
    )


def debye_function(
    x_array: np.array, prec: float = 1e-12, nth_bernoulli: int = 100
) -> np.array:
    r"""Calculates the debye function with n = 3 using one of two series expansions. Valid for |x|<2𝜋 and n≥1.

    For -2pi < x < 0.7𝜋:
    D_3(x) = 1 - \frac{3}{8}x + 3 \sum_{k=1}^{\infty} \frac{B_{2k}}{(2k+3) \Gamma(2k+1)} x^{2k}
    Reference: Eq. (4) by Gonzalez et al. (https://doi.org/10.3390/math10101745)

    For x >= 0.7𝜋:
    D(x) = \frac{\pi^4}{5x^3} - 3 \sum_{k=1}^{\infty} \frac{1}{k} \left(1 + \frac{3}{kx} + \frac{6}{k^2x^2} + \frac{6}{k^3x^3}\right) e^{-kx}
    Reference: Eq. (17) by Khishchenko (https://doi.org/10.20948/MATHMONTIS-2020-49-8)

    Other reference:
    Abramowitz, M. and Stegun, I.A. eds., 1968. Handbook of mathematical functions with formulas, graphs, and mathematical tables (Vol. 55).

    Args:
        x_array: array of input values for the debye function
        prec: Precision. Terminates the series expansion when the absolute value of the term is less than prec.
        nth_bernoulli: Determines the nth Bernoulli number to calculate. A list of Bernoulli numbers is generated prior to calculating the series
        expansion. There should be no reason to change this value under normal circumstances.

    Raises:
        ValueError: If the precision is not between 0 and 1
        ValueError: If x < -2𝜋
        IndexError: If the bernoulli number at index 2k is not available. This indicates slow convergence of the Debye function series expansion.
        If you wish to calculate values for x < -𝜋, convergence may be slow and you may need to increase nth_bernoulli.

    Returns:
        np.array: The value of the debye function evaluated at each x in x_array
    """

    if not 0 < prec < 1:
        raise ValueError("The precision must be between 0 and 1")

    result = np.zeros_like(x_array)
    bern_list = bernoulli(nth_bernoulli)  # 2*k must be less than 100

    for i, x in enumerate(x_array):
        term = 1  # Ensures the while loop runs at least once
        k = 1

        if x >= 0.7 * np.pi:
            debye_value = np.pi**4 / (5 * x**3)
            while abs(term) > prec:
                term = -3 * (
                    1
                    / k
                    * (1 + 3 / (k * x) + 6 / (k**2 * x**2) + 6 / (k**3 * x**3))
                    * np.exp(-k * x)
                )
                debye_value += term
                k += 1
            result[i] = debye_value

        elif -2 * np.pi < x < 0.7 * np.pi:
            debye_value = 1 - 3 / 8 * x
            while abs(term) > prec:
                try:
                    term = 3 * (
                        bern_list[2 * k]
                        / ((2 * k + 3) * gamma(2 * k + 1))
                        * x ** (2 * k)
                    )
                except IndexError:
                    raise IndexError(
                        f"IndexError: the bernoulli number at index {2*k} is not available. This indicates slow convergence of the Debye function series expansion. \
                            If you wish to calculate values for x < -𝜋, convergence may be slow and you may need to increase nth_bernoulli."
                    )
                debye_value += term
                k += 1
            result[i] = debye_value

        else:
            raise ValueError(
                "The debye function series expansions used are only valid for x > -2𝜋"
            )

    return result


def vibrational_energy(temperature: float, theta: float, number_of_atoms) -> float:
    """Evaluates the debye function at x = theta/temperature, then calculates the vibrational energy in eV.

    Args:
        temperature : Temperature in Kelvin
        theta: Debye temperature in Kelvin
        number_of_atoms: Number of atoms in the cell

    Returns:
        float: Vibrational energy in eV
    """

    zero_temp_mask = temperature == 0
    non_zero_temp_mask = temperature > 0

    e_vib = np.zeros_like(temperature)
    debye_value = np.zeros_like(temperature)

    e_vib[zero_temp_mask] = number_of_atoms * (9 / 8 * BOLTZMANN_CONSTANT * theta)

    debye_value[non_zero_temp_mask] = debye_function(
        theta / temperature[non_zero_temp_mask]
    )

    e_vib = (
        number_of_atoms
        * BOLTZMANN_CONSTANT
        * (
            3 * temperature[non_zero_temp_mask] * debye_value[non_zero_temp_mask]
            + 9 / 8 * theta
        )
    )
    return e_vib


def vibrational_entropy(temperature: float, theta: float, number_of_atoms) -> float:
    """Evaluates the debye function at x = theta/temperature, then calculates the vibrational entropy in eV/K.

    Args:
        temperature : Temperature in Kelvin
        theta: Debye temperature in Kelvin
        number_of_atoms: Number of atoms in the cell

    Returns:
        float: Vibrational entropy in eV/K

    """

    zero_temp_mask = temperature == 0
    non_zero_temp_mask = temperature > 0

    s_vib = np.zeros_like(temperature)
    x = np.zeros_like(temperature)
    debye_value = np.zeros_like(temperature)

    s_vib[zero_temp_mask] = 0

    x[non_zero_temp_mask] = theta / temperature[non_zero_temp_mask]
    debye_value[non_zero_temp_mask] = debye_function(x[non_zero_temp_mask])

    s_vib[non_zero_temp_mask] = (
        3
        * number_of_atoms
        * BOLTZMANN_CONSTANT
        * (
            4 / 3 * debye_value[non_zero_temp_mask]
            - np.log(1 - np.exp(-x[non_zero_temp_mask]))
        )
    )

    return s_vib


def vibrational_helmholtz_energy(
    temperature: np.ndarray | float, theta: float, number_of_atoms
) -> float:
    """Evaluates the debye function at x = theta/temperature, then calculates the vibrational Helmholtz energy in eV.

    Args:
        temperature : Temperature in Kelvin
        theta: Debye temperature in Kelvin
        number_of_atoms: Number of atoms in the cell

    Returns:
        float: Vibrational Helmholtz energy in eV
    """

    zero_temp_mask = temperature == 0
    non_zero_temp_mask = temperature > 0

    f_vib = np.zeros_like(temperature)
    x = np.zeros_like(temperature)
    debye_value = np.zeros_like(temperature)

    # Zero point energy
    f_vib[zero_temp_mask] = number_of_atoms * (9 / 8 * BOLTZMANN_CONSTANT * theta)

    x[non_zero_temp_mask] = theta / temperature[non_zero_temp_mask]
    debye_value[non_zero_temp_mask] = debye_function(x[non_zero_temp_mask])

    f_vib[non_zero_temp_mask] = number_of_atoms * (
        9 / 8 * BOLTZMANN_CONSTANT * theta
        + BOLTZMANN_CONSTANT
        * temperature[non_zero_temp_mask]
        * (
            3 * np.log(1 - np.exp(-x[non_zero_temp_mask]))
            - debye_value[non_zero_temp_mask]
        )
    )

    return f_vib


def vibrational_heat_capacity(
    temperature: np.ndarray | float, theta: float, number_of_atoms
) -> float:
    """Evaluates the debye function and its derivative at x = theta/temperature, then calculates the vibrational heat capacity in eV/K.
    The formula is taken from Eq. (13) from Khishchenko (https://doi.org/10.20948/MATHMONTIS-2020-49-8).

    Args:
        temperature : Temperature in Kelvin
        theta: Debye temperature in Kelvin
        number_of_atoms: Number of atoms in the cell

    Returns:
        float: Vibrational heat capacity in eV/K
    """

    zero_temp_mask = temperature == 0
    non_zero_temp_mask = temperature > 0

    cv_vib = np.zeros_like(temperature)
    x = np.zeros_like(temperature)
    debye_value = np.zeros_like(temperature)

    cv_vib[zero_temp_mask] = 0

    x[non_zero_temp_mask] = theta / temperature[non_zero_temp_mask]
    debye_value[non_zero_temp_mask] = debye_function(x[non_zero_temp_mask])

    cv_vib[non_zero_temp_mask] = (
        3
        * number_of_atoms
        * BOLTZMANN_CONSTANT
        * (
            4 * debye_value[non_zero_temp_mask]
            - 3 * x[non_zero_temp_mask] / (np.exp(x[non_zero_temp_mask]) - 1)
        )
    )
    return cv_vib


"""
equation of state
"""
def murnaghan_equation(
    volume: float | np.ndarray, V0: float, E0: float, B: float, BP: float
) -> float | np.ndarray:
    """Murnaghan EOS

    Args:
        volume (float | np.ndarray): input volume
        V0 (float): equilibrium volume
        E0 (float): equilibrium energy
        B (float): bulk modulus
        BP (float): derivative of bulk modulus with respect to pressure

    Returns:
        float | np.ndarray: energy
    """

    energy = (
        E0
        - (B * V0) / (BP - 1)
        + (B * volume / BP) * (1 + (V0 / volume) ** BP / (BP - 1))
    )
    return energy




In [151]:
# define run_demo()

def run_demo(
    v0,
    e0,
    b0,
    b0p,
    volumes,
    temperatures,
    atomic_mass, # typically a geometric or log average atomic mass in daltons
    number_of_atoms,
    gruneisen_x = 1, # small adjustment value 1 for low temperature, 2/3 for high temperature
    scaling_factor = 0.617, # scaling factor determined empiraically for FCC metals. Can also be determined from elastic constants
    
):
    SPEED = 776.024533 # to meters per second
    eos_parameters = (v0, e0, b0, b0p)
    density = (atomic_mass*number_of_atoms/volumes)
    sound_speed = (b0/density)**(1/2)*SPEED
    print(f'The speed of sound in this material is {sound_speed} m/s at volume {volumes}')

    # calculate the energy as function of volume from the given parameters
    mid_vol_index= len(volumes)//2
    mid_vol = volumes[mid_vol_index]
    ev_vols = np.linspace(0.9*mid_vol,1.10*mid_vol, 1001)
    energies = np.zeros(len(ev_vols))
    for vol in ev_vols:
        energy = murnaghan_equation(vol, v0, e0, b0, b0p)
        energies[np.where(ev_vols == vol)] = energy
    


    # plot the curve
    ev_fig = go.Figure()
    ev_fig.add_trace(go.Scatter(
        x=ev_vols,
        y=energies,
        mode='lines'
    ))
    ev_fig.show()
    
    # calculate the gruneisen parameter
    gru_par = gruneisen_parameter(b0p, gruneisen_x)

    theta = debye_temperature(
        volumes,
        eos_parameters,
        atomic_mass,
        gru_par,
        scaling_factor
    )

    s_vib_v_t = np.zeros((len(volumes), len(temperatures)))
    f_vib_v_t = np.zeros((len(volumes), len(temperatures)))
    cv_vib_v_t = np.zeros((len(volumes), len(temperatures)))

    for i, volume in enumerate(volumes):
        s_vib = vibrational_entropy(temperatures, theta[i], number_of_atoms)
        f_vib = vibrational_helmholtz_energy(
            temperatures, theta[i], number_of_atoms
        )
        cv_vib = vibrational_heat_capacity(temperatures, theta[i], number_of_atoms)
        s_vib_v_t[i, :] = s_vib
        f_vib_v_t[i, :] = f_vib
        cv_vib_v_t[i, :] = cv_vib
    
    # plot the entropy at volumes 90,95,100,105,110 A^3 as a function of temperature
    selected_volumes = volumes
    # s_fig = go.Figure()
    # for i, volume in enumerate(selected_volumes):
    #     s_fig.add_trace(go.Scatter(
    #         x=temperatures,
    #         y=s_vib_v_t[np.where(volumes == volume)][0],
    #         mode='lines',
    #         name=f'{volume} A^3'
    #     ))
    # s_fig.update_layout(title='Vibrational Entropy vs Temperature', xaxis_title='Temperature (K)', yaxis_title='Entropy (eV/K)')
    # s_fig.show()

    # # plot the free energy
    # f_fig = go.Figure()
    # for i, volume in enumerate(selected_volumes):
    #     f_fig.add_trace(go.Scatter(
    #         x=temperatures,
    #         y=f_vib_v_t[np.where(volumes == volume)][0],
    #         mode='lines',
    #         name=f'{volume} A^3'
    #     ))
    # f_fig.update_layout(
    #     title='Vibrational Free Energy vs Temperature',
    #     xaxis_title='Temperature (K)',
    #     yaxis_title='Free Energy (eV)'
    # )
    # f_fig.show()

    # plot the heat capacity
    cv_fig = go.Figure()
    for i, volume in enumerate(selected_volumes):
        cv_fig.add_trace(go.Scatter(
            x=temperatures,
            y=cv_vib_v_t[np.where(volumes == volume)][0],
            mode='lines',
            name=f'{volume} A^3'
        ))
    cv_fig.update_layout(
        title='Vibrational Heat Capacity vs Temperature',
        xaxis_title='Temperature (K)',
        yaxis_title='Heat Capacity (eV/K)'
    )
    cv_fig.show()

    


# Questions
1. How does pressure affect the heat capacity?
2. How does temperature affect the heat capcity?
3. How does atomic mass affect the heat capacity?
4. How does bulk modulus affect the heat capcity?
5. How does the pressure derivative of the bulk modulus affect the heat capcity?

## Step 1.

In the two cells below, adjust the values as desired

In [152]:
volumes = np.linspace(65.89*0.9, 65.89*1.1, 5)
temperatures = np.linspace(0, 1000, 101)
atomic_mass = 26.98 # typically a geometric or log average atomic mass in daltons
gruneisen_x = 2/3 # small adjustment value 1 for low temperature, 2/3 for high temperature
scaling_factor = 0.617 # scaling factor determined empiraically for FCC metals. Can also be determined from elastic constants
number_of_atoms = 4


In [153]:
# parameters for the four parameter Birch Murnaghan (BM4) equation of state (EOS) 
v0 = 65.89 # equilibrium volume in A^3
e0 = 0.0 # energy at v0
b0 = 69 # bulk modulus at v0 in GPa
b0p = 4.14 # pressure derivative of the bulk modulus at v0


## Step 2.
Run the cell below

In [154]:
run_demo(
    v0=v0,
    e0=e0,
    b0=b0,
    b0p=b0p,
    volumes=volumes,
    temperatures=temperatures,
    atomic_mass=atomic_mass,
    gruneisen_x=gruneisen_x,
    scaling_factor=scaling_factor,
    number_of_atoms=number_of_atoms
)

The speed of sound in this material is [4778.37287759 4909.31144633 5036.84726758 5161.23259666 5282.68998112] m/s at volume [59.301  62.5955 65.89   69.1845 72.479 ]
